# <font style="color:blue">Project 2: Kaggle Competition - Classification</font>

#### Maximum Points: 100

<div>
    <table>
        <tr><td><h3>Sr. no.</h3></td> <td><h3>Section</h3></td> <td><h3>Points</h3></td> </tr>
        <tr><td><h3>1</h3></td> <td><h3>Data Loader</h3></td> <td><h3>10</h3></td> </tr>
        <tr><td><h3>2</h3></td> <td><h3>Configuration</h3></td> <td><h3>5</h3></td> </tr>
        <tr><td><h3>3</h3></td> <td><h3>Evaluation Metric</h3></td> <td><h3>10</h3></td> </tr>
        <tr><td><h3>4</h3></td> <td><h3>Train and Validation</h3></td> <td><h3>5</h3></td> </tr>
        <tr><td><h3>5</h3></td> <td><h3>Model</h3></td> <td><h3>5</h3></td> </tr>
        <tr><td><h3>6</h3></td> <td><h3>Utils</h3></td> <td><h3>5</h3></td> </tr>
        <tr><td><h3>7</h3></td> <td><h3>Experiment</h3></td><td><h3>5</h3></td> </tr>
        <tr><td><h3>8</h3></td> <td><h3>TensorBoard Dev Scalars Log Link</h3></td> <td><h3>5</h3></td> </tr>
        <tr><td><h3>9</h3></td> <td><h3>Kaggle Profile Link</h3></td> <td><h3>50</h3></td> </tr>
    </table>
</div>


## <font style="color:green">1. Data Loader [10 Points]</font>

In this section, you have to write a class or methods, which will be used to get training and validation data loader.

You need to write a custom dataset class to load data.

**Note; There is   no separate validation data. , You will thus have to create your own validation set, by dividing the train data into train and validation data. Usually, we do 80:20 ratio for train and validation, respectively.**


For example:

```python
class KenyanFood13Dataset(Dataset):
    """
    
    """
    
    def __init__(self, *args):
    ....
    ...
    
    def __getitem__(self, idx):
    ...
    ...
    
    
```

```
def get_data(args1, *agrs):
    ....
    ....
    return train_loader, test_loader
```

In [1]:
import os
import shutil
import sys
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch import optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torchvision.models as models
from torchvision.models import resnet50, EfficientNet_B1_Weights

import pytorch_lightning as pl
import torchmetrics
from torchmetrics import Metric
from torchmetrics import AveragePrecision
from torchmetrics.classification import accuracy
from torchmetrics.classification import MulticlassConfusionMatrix
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning import callbacks
from lightning.pytorch.callbacks import ModelCheckpoint

from dataclasses import dataclass

from PIL import Image

In [2]:
csv_path = './opencv-pytorch-dl-course-classification/train.csv'
df = pd.read_csv(csv_path)
num_classes = len(df['class'].unique())

In [3]:
def img_preprocess_transforms():
    preprocess = transforms.Compose([
                transforms.Resize(256),
                transforms.CenterCrop(224),
                transforms.ToTensor()
            ])
    return preprocess

In [4]:
def get_mean_std(data_root, num_classes, class_to_idx, training_testing,num_workers=2, batch_size=32 ):
    # Name of the dataset we need to extract the info from
    
    if training_testing == 'train':
        dataset = KenyanFood13DatasetTrain(data_root, training_testing, num_classes, transform=img_preprocess_transforms(), class_to_idx = class_to_idx)
    elif training_testing == 'test':
         dataset = KenyanFood13DatasetTest(data_root, training_testing, num_classes, transform=img_preprocess_transforms())

    dataloader = DataLoader(dataset, batch_size=batch_size, num_workers=num_workers)

    batch_mean = torch.zeros(3)
    batch_mean_sqrd = torch.zeros(3)

    for batch_data, _ in dataloader:
        batch_mean += batch_data.mean(dim=(0, 2, 3))
        batch_mean_sqrd += (batch_data ** 2).mean(dim=(0, 2, 3))

    mean = batch_mean / len(dataloader)
    var = (batch_mean_sqrd / len(dataloader)) - (mean ** 2)
    std = var ** 0.5

    print('mean: {}, std: {}'.format(mean, std))
    return mean, std


In [5]:
class KenyanFood13DatasetTrain(Dataset):
    def __init__(self, data_root, train: str, num_classes, transform,
                 class_to_idx
                 ):
        super().__init__()
        self.data_root = data_root
        self.train = train
        self.num_classes = num_classes
        self.transform = transform
        self.class_to_idx = class_to_idx

        csv_path = './opencv-pytorch-dl-course-classification/train.csv' 
        df = pd.read_csv(csv_path)
        print(csv_path)

        # generate data frames
        if self.train != 'test':
            train_df, val_df = train_test_split(df, test_size = 0.2, train_size = 0.8, random_state=42, shuffle = True)
            if self.train == 'train':
                df = train_df 
            elif self.train == 'val':
                df = val_df

        # Join the data root path with the training/training, this creates a var in memory with the training data root
        # self.img_dir = os.path.join(self.data_root, self.train, self.train)
        self.dict_init(df)

    def dict_init(self, df):
        # Initialize the data dictionary
        if self.train != 'test':
            self.data_dict = {
                'image_path': [],
                'label': [] 
            }

        if self.train != 'test':
            for id in os.listdir(self.data_root):
                if id.endswith(".jpg") or id.endswith(".png"):
                    image_id = id.split('.')[0]
                    matching_rows = df[df['id'] == int(image_id)]
                    if not matching_rows.empty:
                        class_label = matching_rows.iloc[0]['class']
                        img_path = os.path.join(self.data_root, id)
                        self.data_dict['image_path'].append(img_path)
                        self.data_dict['label'].append(class_label)
                    
    def __numClasses__(self):
        return int(self.num_classes)

    def __len__(self):
        """
        return length of the dataset
        """
        return len(self.data_dict['image_path'])

    def __getitem__(self, idx):
        """
        For a given index, return images with resizing and preprocessing.
        """
        image = Image.open(self.data_dict['image_path'][idx]).convert("RGB")
        if self.transform is not None:
            image = self.transform(image)
        if self.train != 'test':
            class_name = self.data_dict['label'][idx]
            target = self.class_to_idx[class_name]
            return image, target


In [6]:
class KenyanFood13DatasetTest(Dataset):
    def __init__(self, data_root, train: str, num_classes, transform):
        super().__init__()
        self.data_root = data_root
        self.train = train
        self.num_classes = num_classes
        self.transform = transform

        csv_path = './opencv-pytorch-dl-course-classification/test.csv'
        df = pd.read_csv(csv_path)
        print(csv_path)

        # Join the data root path with the training/training, this creates a var in memory with the training data root
        # self.img_dir = os.path.join(self.data_root, self.train, self.train)
        self.dict_init()

    def dict_init(self):
        # Initialize the data dictionary for the test set
        self.data_dict = {
            'image_path': [],
        }

        for id in os.listdir(self.data_root):
            if id.endswith(".jpg") or id.endswith(".png"):
                img_path = os.path.join(self.data_root, id)
                self.data_dict['image_path'].append(img_path)
                    
    def __len__(self):
        """
        return length of the dataset
        """
        return len(self.data_dict['image_path'])

    def __getitem__(self, idx):
        """
        For a given index, return images with resizing and preprocessing.
        """
        image = Image.open(self.data_dict['image_path'][idx]).convert("RGB")
        if self.transform is not None:
            image = self.transform(image)

        image_id = self.data_dict['image_path'][idx]
        return image, image_id


In [7]:
class KenyanFoodDataModuleTrain(pl.LightningDataModule):
    def __init__(self, data_root, batch_size, num_workers, num_classes, augmentation = False):
        super().__init__()
        self.data_root = data_root
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.num_classes = num_classes

        # Create the class-to-index mapping
        self.class_to_idx = self._create_class_to_idx_mapping()

        self.preprocess = img_preprocess_transforms()
        self.mean, self.std = get_mean_std(self.data_root, self.num_classes, self.class_to_idx, training_testing="train")

        if augmentation:
            additional_augmentations = transforms.Compose([
                transforms.RandomHorizontalFlip(),
                transforms.RandomRotation(degrees=10),
                transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Adjust color,
                ])
            self.transform = transforms.Compose([
                additional_augmentations,
                self.preprocess,
                transforms.Normalize(self.mean, self.std)
            ])
        else:
            self.transform = transforms.Compose([
                self.preprocess,
                transforms.Normalize(self.mean, self.std)
            ])

    def _create_class_to_idx_mapping(self):
        # Read your CSV file and obtain unique class names
        unique_class_names = pd.read_csv('./opencv-pytorch-dl-course-classification/train.csv')['class'].unique()

        # Create a mapping from class names to integer indices
        class_to_idx = {class_name: idx for idx, class_name in enumerate(unique_class_names)}
        return class_to_idx

    def setup(self, stage):
        # multiple gpus
            
        self.train_dataset = KenyanFood13DatasetTrain(data_root=self.data_root,
                                                    train='train',
                                                    num_classes = self.num_classes,
                                                    transform = self.transform,
                                                    class_to_idx = self.class_to_idx
                                                    )
        
        self.val_dataset =   KenyanFood13DatasetTrain(data_root=self.data_root,
                                                    train='val',
                                                    num_classes = self.num_classes ,
                                                    transform = self.transform,
                                                    class_to_idx = self.class_to_idx
                                                    )
    
    def train_dataloader(self):
        return DataLoader(self.train_dataset,
                          batch_size = self.batch_size,
                          num_workers = self.num_workers,
                          shuffle = True
                          )

    def val_dataloader(self):
        return DataLoader(self.val_dataset,
                          batch_size = self.batch_size,
                          num_workers = self.num_workers,
                          shuffle = False
                          )

    def __numClasses__(self):
        return self.train_dataset.__numClasses__()

In [8]:
class KenyanFoodDataModuleTest(pl.LightningDataModule):
    def __init__(self, data_root, batch_size, num_workers, augmentation = False):
        super().__init__()
        self.data_root = data_root
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.num_classes = num_classes

        self.preprocess = img_preprocess_transforms()
        self.mean, self.std = get_mean_std(self.data_root, self.num_classes, 0, training_testing="test")

        self.transform = transforms.Compose([
            self.preprocess,
            transforms.Normalize(self.mean, self.std)
        ])

    def setup(self, stage):
        # multiple gpus
    
        self.test_dataset =  KenyanFood13DatasetTest(data_root=self.data_root,
                                                    train='test',
                                                    num_classes = self.num_classes ,
                                                    transform = transforms.Compose([
                                                            self.preprocess,
                                                            transforms.Normalize(self.mean, self.std)
                                                        ]),
                                        
                                                    )

    def test_dataloader(self):
        return DataLoader(self.test_dataset,
                          batch_size = self.batch_size,
                          num_workers = self.num_workers,
                          shuffle = False
                          )

## <font style="color:green">2. Configuration [5 Points]</font>

**Define your configuration here.**

For example:


```python
@dataclass
class TrainingConfiguration:
    '''
    Describes configuration of the training process
    '''
    batch_size: int = 10
    epochs_count: int = 50  
    init_learning_rate: float = 0.1  # initial learning rate for lr scheduler
    log_interval: int = 5  
    test_interval: int = 1  
    data_root: str = "/kaggle/input/pytorch-opencv-course-classification/"
    num_workers: int = 2  
    device: str = 'cuda'  
    
```

In [34]:
from dataclasses import dataclass, field

@dataclass
class TrainingConfiguration:
    '''
    Describes configuration of the training process
    '''
    augmentation_train: bool = True
    augmentation_test: bool = False
    batch_size: int = 35
    min_epochs: int = 0
    max_epochs: int = 58
    lr: float = 0.0001  # initial learning rate for lr scheduler
    data_root: str = "./opencv-pytorch-dl-course-classification/images/images" # RMEMBER TO CHANGE THIS ACCORDING TO THE DIRECTORY WE HAVE BY DEFAULT IN EACH ENV
    num_workers: int = 2
    device: int = 0
    step_training: str = 'train'
    step_test: str = 'test'
    precision: str = "16-mixed"
    accelerator: str = "cuda" # Available names are: auto, cpu, cuda, mps, tpu.


In [10]:
if torch.cuda.is_available():
    # Get the number of available GPUs
    num_gpus = torch.cuda.device_count()
    print(f"Number of available GPUs: {num_gpus}")

    # Get the index of the current GPU (if you have only one GPU)
    current_gpu_index = torch.cuda.current_device()
    print(f"Current GPU index: {current_gpu_index}")

    # Get the name of the current GPU
    current_gpu_name = torch.cuda.get_device_name(current_gpu_index)
    print(f"Current GPU name: {current_gpu_name}")
else:
    print("No GPU available.")

Number of available GPUs: 1
Current GPU index: 0
Current GPU name: NVIDIA GeForce GTX 1060 3GB


## <font style="color:green">3. Evaluation Metric [10 Points]</font>

**Define methods or classes that will be used in model evaluation. For example, accuracy, f1-score etc.**

In [11]:
class MyAccuracy():
    def __init__(self, num_classes, device: TrainingConfiguration.device):
        super().__init__()
        self.device = device
        self.num_classes = num_classes
        self.accuracy = torchmetrics.Accuracy(task='multiclass', num_classes = self.num_classes).to(self.device)
        self.f1_score = torchmetrics.F1Score(task='multiclass', num_classes = self.num_classes).to(self.device)


    def accuracyMetric(self, pred, target):
        pred = pred.to(self.device)
        target = target.to(self.device)
        accuracy = self.accuracy(pred, target)
        return accuracy

    def f1Metric(self, pred, target):
        pred = pred.to(self.device)
        target = target.to(self.device)
        f1_score = self.f1_score(pred, target)
        return f1_score

## <font style="color:green">4. Train and Validation [5 Points]</font>


**Write the methods or classes to be used for training and validation.**

In [12]:
# TRAINING AND VAL STEPS WERE INCLUDED IN SECTION 5

## <font style="color:green">5. Model [5 Points]</font>

**Define your model in this section.**

**You are allowed to use any pre-trained model.**

In [13]:
def pretrained_model(num_classes, dropout_rate):
    new_model = models.efficientnet_b1(weights=EfficientNet_B1_Weights.IMAGENET1K_V2)
    
    new_model.classifier[1] = nn.Linear(new_model.classifier[1].in_features, num_classes)

    return new_model

In [14]:
class NN(pl.LightningModule):  # here nn.Module is replaced by LightningModule
    def __init__(self, num_classes, class_to_idx, device ,learning_rate = 0.1, dropout = 0.25):
        super().__init__()
        self.num_classes = num_classes
        self.lr = learning_rate
        self.device_id = device
        self.dropout = dropout
        self.confusion_matrix = ConfMatrixUtil(self.num_classes, self.device_id)
        self.class_to_idx = class_to_idx

        # Set acc variables and f1 vars
        self.train_acc_f1 = MyAccuracy(self.num_classes, self.device_id)
        self.valid_acc_f1 = MyAccuracy(self.num_classes, self.device_id)
        self.test_acc_f1 = MyAccuracy(self.num_classes, self.device_id)

        # Set loss variables
        average_meter = AveragePrecision(task = "multiclass", num_classes = self.num_classes)
        self.train_loss = average_meter.clone()
        self.valid_loss = average_meter.clone()
        self.test_loss = average_meter.clone()
        
        self.results_dict = {
            'image_ids': [],
            'predictions': []
        }
        
        # Save the arguments as hyperparameters.
        self.save_hyperparameters()
        self.model = pretrained_model(self.num_classes, self.dropout)

    def forward(self, data):
        output = self.model(data)
        return output

    def training_step(self, batch, batch_idx):
        target, prob, loss = self._common_step(batch, batch_idx)
        pred = prob.data.max(dim=1)[1]
        acc = self.train_acc_f1.accuracyMetric(pred, target)
        f1 = self.train_acc_f1.f1Metric(pred, target)
        self.confusion_matrix.update(pred, target)
        # self.train_loss(loss)
        self.log("train/batch_loss", loss, prog_bar=True)
        self.log("train/batch_acc", acc, prog_bar=True)
        self.log("train/batch_f1", f1, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        target, prob, loss = self._common_step(batch, batch_idx)
        pred = torch.argmax(prob, dim=1)
        acc = self.valid_acc_f1.accuracyMetric(pred, target)
        f1 = self.valid_acc_f1.f1Metric(pred, target)
        self.confusion_matrix.update(pred, target)
        # self.valid_loss(loss)
        self.log("valid/batch_loss", loss, prog_bar=True)
        self.log("valid/batch_acc", acc, prog_bar=True)
        self.log("valid/batch_f1", f1, prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx):
        images, image_ids= batch
        output = self(images)
        # Obtain predictions
        prob = F.softmax(output, dim=1)
        preds = torch.argmax(prob, dim=1)
        for image_id, pred in zip(image_ids, preds):
            # Extract the image_id without file extensions
            image_id = image_id.split('/')[-1].split('.')[0]
            # Check if the image_id is not already present in the dictionary
            if image_id not in self.results_dict['image_ids']:
                self.results_dict['image_ids'].append(image_id)
                self.results_dict['predictions'].append(pred.item())
            
    def _common_step(self, batch, batch_idx):
        data, target = batch
        count = 0
        output = self(data)
        loss = F.cross_entropy(output, target)
        prob = F.softmax(output, dim=1)
        return target, prob, loss

    def on_train_epoch_end(self):
        """Calculate epoch level metrics for the train set"""
        conf_mat = self.confusion_matrix.compute()
        self.log("train/confusion_matrix", conf_mat, prog_bar=True)
        self.log("step", self.current_epoch, prog_bar=True)

    def on_validation_epoch_end(self):
        """Calculate epoch level metrics for the validation set"""
        conf_mat = self.confusion_matrix.compute()
        self.log("valid/confusion_matrix", conf_mat, prog_bar=True)
        self.log("step", self.current_epoch, prog_bar=True)

    def test_results(self):
        return self.results_dict

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), self.lr)

## <font style="color:green">6. Utils [5 Points]</font>

**Define those methods or classes, which have  not been covered in the above sections.**

In [15]:
# REMINDER: Metrics and utilities will be used in the TrainingValidation and Test classes
class ConfMatrixUtil():
    def __init__(self, num_classes, device):
        self.device = device
        self.num_classes = num_classes
        self.metric = MulticlassConfusionMatrix(num_classes=self.num_classes).to(self.device)
        self.preds = None
        self.targets = None

    def update(self, pred, target):
        self.preds = pred.to(self.device)
        self.targets = target.to(self.device)

    def compute(self):
        # Calculate the confusion matrix
        confusion_matrix = self.metric(self.preds, self. targets)
        accuracy_score =torch.sum(torch.diag(confusion_matrix)) / torch.sum(confusion_matrix)
        return accuracy_score

## <font style="color:green">7. Experiment [5 Points]</font>

**Choose your optimizer and LR-scheduler and use the above methods and classes to train your model.**

In [16]:
dm_train = KenyanFoodDataModuleTrain(data_root = TrainingConfiguration.data_root,
                          batch_size = TrainingConfiguration.batch_size,
                          num_workers = TrainingConfiguration.num_workers,
                          num_classes = num_classes,
                          augmentation = TrainingConfiguration.augmentation_train
                          )

./opencv-pytorch-dl-course-classification/train.csv
mean: tensor([0.5763, 0.4620, 0.3453]), std: tensor([0.2702, 0.2740, 0.2827])


In [17]:
dm_test = KenyanFoodDataModuleTest(data_root = TrainingConfiguration.data_root,
                          batch_size = TrainingConfiguration.batch_size,
                          num_workers = TrainingConfiguration.num_workers,
                          augmentation = TrainingConfiguration.augmentation_train
                          )

./opencv-pytorch-dl-course-classification/test.csv
mean: tensor([0.5767, 0.4626, 0.3467]), std: tensor([0.2698, 0.2738, 0.2827])


In [18]:
class_to_idx = dm_train._create_class_to_idx_mapping()
idx_to_class = {index: class_name for class_name, index in class_to_idx.items()}

In [35]:
logger = TensorBoardLogger("tb_logs", name="Kenyan_Food")

trainer = pl.Trainer(accelerator=TrainingConfiguration.accelerator, 
                    logger=logger,
                    # devices = TrainingConfiguration.device,
                    min_epochs=TrainingConfiguration.min_epochs,
                    max_epochs=TrainingConfiguration.max_epochs,
                    precision=TrainingConfiguration.precision,
                    )

model = NN(learning_rate = TrainingConfiguration.lr, num_classes = num_classes, device = TrainingConfiguration.device, class_to_idx = class_to_idx)
print(model)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


NN(
  (train_loss): MulticlassAveragePrecision()
  (valid_loss): MulticlassAveragePrecision()
  (test_loss): MulticlassAveragePrecision()
  (model): EfficientNet(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): SiLU(inplace=True)
      )
      (1): Sequential(
        (0): MBConv(
          (block): Sequential(
            (0): Conv2dNormActivation(
              (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
              (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): SiLU(inplace=True)
            )
            (1): SqueezeExcitation(
              (avgpool): AdaptiveAvgPool2d(output_size=1)
              (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
              (f

In [36]:
trainer.fit(model, dm_train)

./opencv-pytorch-dl-course-classification/train.csv
./opencv-pytorch-dl-course-classification/train.csv


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type                       | Params
----------------------------------------------------------
0 | train_loss | MulticlassAveragePrecision | 0     
1 | valid_loss | MulticlassAveragePrecision | 0     
2 | test_loss  | MulticlassAveragePrecision | 0     
3 | model      | EfficientNet               | 6.5 M 
----------------------------------------------------------
6.5 M     Trainable params
0         Non-trainable params
6.5 M     Total params
26.119    Total estimated model params size (MB)


Epoch 0: 100%|█| 150/150 [04:09<00:00,  0.60it/s, v_num=9, train/batch_loss=1.800, train/batch_acc=0
Validation: |                                                                 | 0/? [00:00<?, ?it/s]
Validation DataLoader 0: 100%|██████████████████████████████████████| 38/38 [00:56<00:00,  0.67it/s]
Epoch 1: 100%|█| 150/150 [03:53<00:00,  0.64it/s, v_num=9, train/batch_loss=1.610, train/batch_acc=0
Validation: |                                                                 | 0/? [00:00<?, ?it/s]
Validation DataLoader 0: 100%|██████████████████████████████████████| 38/38 [00:53<00:00,  0.71it/s]
Epoch 2: 100%|█| 150/150 [03:52<00:00,  0.64it/s, v_num=9, train/batch_loss=1.010, train/batch_acc=0
Validation: |                                                                 | 0/? [00:00<?, ?it/s]
Validation DataLoader 0: 100%|██████████████████████████████████████| 38/38 [00:54<00:00,  0.70it/s]
Epoch 3: 100%|█| 150/150 [03:50<00:00,  0.65it/s, v_num=9, train/batch_loss=1.600, train/ba

`Trainer.fit` stopped: `max_epochs=58` reached.


Epoch 57: 100%|█| 150/150 [04:55<00:00,  0.51it/s, v_num=9, train/batch_loss=0.00936, train/batch_ac


In [37]:
trainer.validate(model, dm_train)

./opencv-pytorch-dl-course-classification/train.csv
./opencv-pytorch-dl-course-classification/train.csv


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation DataLoader 0: 100%|██████████████████████████████████████| 38/38 [00:54<00:00,  0.70it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
          step                     58.0
     valid/batch_acc        0.7591742873191833
     valid/batch_f1         0.7591742873191833
    valid/batch_loss        1.2615362405776978
 valid/confusion_matrix     0.8461538553237915
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'valid/batch_loss': 1.2615362405776978,
  'valid/batch_acc': 0.7591742873191833,
  'valid/batch_f1': 0.7591742873191833,
  'valid/confusion_matrix': 0.8461538553237915,
  'step': 58.0}]

In [38]:
trainer.test(model, dm_test) # Modifications done to the tes_step in order to return a dict with 'predictions' and  'image_ids' keys

./opencv-pytorch-dl-course-classification/test.csv


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|███████████████████████████████████████| 234/234 [01:10<00:00,  3.30it/s]


[{}]

In [39]:
results = model.test_results()

In [40]:
# Generate submission.csv file
print(len(results['predictions']))
print(len(results['image_ids']))
predictions = results['predictions']
image_ids = results['image_ids']

# Map Predictions to Classes
predicted_classes = [idx_to_class[pred] for pred in predictions]

# Create Submission DataFrame
submission_df = pd.DataFrame({'id': image_ids, 'class': predicted_classes})

# Save to CSV
submission_df.to_csv('test_results.csv', index=False)

8174
8174


In [41]:
# Generate submission.csv file
matching_ids = df['id'].tolist()

df_2 = pd.read_csv('test_results.csv')

csv_filtered = df_2[~df_2['id'].isin(matching_ids)]

csv_filtered.to_csv('submission.csv', index = False)

## <font style="color:green">8. TensorBoard Dev Scalars Log Link [5 Points]</font>

**Share your TensorBoard scalars logs link here You can also share (not mandatory) your GitHub link, if you have pushed this project in GitHub.**


For example, [Find Project2 logs here](https://tensorboard.dev/experiment/kMJ4YU0wSNG0IkjrluQ5Dg/#scalars).

In [ ]:
!tensorboard --logdir="tb_logs" --port default

TensorFlow installation not found - running with reduced feature set.

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.14.0 at http://localhost:6006/ (Press CTRL+C to quit)


## <font style="color:green">9. Kaggle Profile Link [50 Points]</font>

**Share your Kaggle profile link  with us here to score , points in  the competition.**

**For full points, you need a minimum accuracy of `75%` on the test data. If accuracy is less than `70%`, you gain  no points for this section.**


**Submit `submission.csv` (prediction for images in `test.csv`), in the `Submit Predictions` tab in Kaggle, to get evaluated for  this section.**

https://www.kaggle.com/agustinpernigotti